<a href="https://colab.research.google.com/github/franciscogazaniga/TCC_Aprendizado_Maquina_Previsao_Energia_Fotovoltaica/blob/main/AED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import

In [ ]:
%%capture
%pip install kaleido
%pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px 
import plotly.graph_objects as go 
import os 
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor


# 1.1 Ambiente

In [ ]:
from symbol import tfpdef


plt.style.use('seaborn-talk')

if not os.path.exists("images"):
    os.mkdir("images")

def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(42)
   tfpdef.random.set_seed(42)
   np.random.seed(42)

# Definir seu diretório aqui
diretorioDados = '/content/drive/MyDrive/TCC/Banco de Dados'
diretorioImagens = '/content/drive/MyDrive/TCC/Imagens'
diretorioModelos = '/content/drive/MyDrive/TCC/Modelos'    

#2. Carregamento dos dados

In [ ]:
# Dados de geração
dtGeneration = pd.read_csv(diretorioDados + '/Plant_2_Generation_Data.csv')

# Dados de clima
dtWeather = pd.read_csv(diretorioDados + '/Plant_2_Weather_Sensor_Data.csv')

In [ ]:
dtGeneration.head()

In [ ]:
dtWeather.head()

In [ ]:
# Une os dados de clima e geração de energia em uma única tabela
dtUnificadaSemTratativa = pd.merge(dtGeneration, dtWeather, how = 'inner', on = 'DATE_TIME')
dtUnificadaSemTratativa.head()

In [ ]:
# Verificação de valores únicos
print(dtUnificadaSemTratativa.nunique())

As colunas "PLANT_ID_x" e "PLANT_ID_y" possuem um valor único e correspondem ao mesmo valor, que indica que os dados se referem a Planta 2, portanto estas colunas serão desnecessárias na análise dos dados e serão excluidas. A coluna "SOURCE_KEY_y" será também excluída, visto que se refere ao sensor de temperatura, cujo é unitário.

In [ ]:
# Exclui colunas "PLANT_ID_x", "PLANT_ID_y" e SOURCE_KEY_y"
dtUnificadaTrativa1 = dtUnificadaSemTratativa.drop(columns=['PLANT_ID_x', 'PLANT_ID_y', 'SOURCE_KEY_y'])

# Renomeia coluna "SOURCE_KEY_x" para "SOURCE_KEY"
dtUnificadaTrativa2 = dtUnificadaTrativa1.rename(columns={'SOURCE_KEY_x': 'SOURCE_KEY'})

In [ ]:
dtUnificadaTrativa2.head()

In [ ]:
# Verifica quantidade de linhas e colunas das tabelas
print(dtGeneration.shape, dtWeather.shape, dtUnificadaTrativa2.shape)

Pode-se constatar que a dtUnificadaTrativa2 contêm a mesma quantidade de linhas que a dtGeneration, tabela com mais dados que a dtWeather. Possui mais dados devido a quantidade de inversores na dtGeneration ser maior do que a quantidade de sensores na dtWeather.

# 3. Análise Exploratória de Dados

In [ ]:
dtUnificadaTrativa2.dtypes

In [ ]:
# Realizada uma cópia dos dados para um dataset onde serão processados os dados
dtUnificadaTrativa3 = dtUnificadaTrativa2.copy()

# Convertendo a coluna 'date' para 'datetime'
dtUnificadaTrativa3['DATE_TIME'] = pd.to_datetime(dtUnificadaTrativa3['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
dtUnificadaTrativa3.dtypes

In [ ]:
dtUnificadaTrativa3.head()

## 3.1 Validação das linhas de dados na amostra

In [ ]:
dtUnificadaTrativa3.describe()

Percebe-se uma grande quantidade de valores iguais a zero para a geração de energia no dataset, visto que o monitoramento continua durante o período da noite, isto faz sentido, no entanto, pode ser que o registro dos inversores tenha sido igual a zero mesmo durante o dia, antes de analisar isto, será verificado se existe algum valor faltante ou duplicado no conjunto de dados.

In [ ]:
# Verificação de valor missing/NA
print(dtUnificadaTrativa3.isna().sum())

Não existem valores faltantes, classificados como NaN, na amostra.

In [ ]:
# Verificar se existem valores duplicados
print(sum(dtUnificadaTrativa3.duplicated()))

Não existem dados duplicados.
Não existir valores faltantes na amostra não indica que não está faltando alguma linha com informação. Visto que para os dados de geração de energia, as aferições são realizadas a cada 15 minutos, e a coleta foi feita por um período de 24 horas durante 34 dias, para 22 inversores, se tem que a quantidade de linhas para este conjunto de dados deveria ser igual a 71.808, e como vimos anteriormente, a tabela de dados possui apenas 67.698 linhas de dados, o que indica que em alguns momentos das aferições não houveram registro de informação.

Para um único inversor, deve haver 3.264 linhas de dados, portanto, a seguir será verificado a quantidade de linhas gerada por cada inversor.

In [ ]:
# Quantidade de instâncias para cada inversor no conjunto de dados
dtUnificadaTrativa3["SOURCE_KEY"].value_counts()

In [ ]:
inversor1 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == '4UPUqMRk7TRMgml']
inversor2 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == '81aHJ1q11NBPMrL']
inversor3 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == '9kRcWv60rDACzjR']
inversor4 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'Et9kgGMDl729KT4']
inversor5 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'IQ2d7wF4YD8zU1Q']
inversor6 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'LYwnQax7tkwH5Cb']
inversor7 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'LlT2YUhhzqhg5Sw']
inversor8 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'Mx2yZCDsyf6DPfv']
inversor9 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'NgDl19wMapZy17u']
inversor10 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'PeE6FRyGXUgsRhN']
inversor11 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'Qf4GUc1pJu5T6c6']
inversor12 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'Quc1TzYxW2pYoWX']
inversor13 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'V94E5Ben1TlhnDV']
inversor14 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'WcxssY2VbP4hApt']
inversor15 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'mqwcsP2rE7J0TFp']
inversor16 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'oZ35aAeoifZaQzV']
inversor17 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'oZZkBaNadn6DNKz']
inversor18 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'q49J1IKaHRwDQnt']
inversor19 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'rrq4fwE8jgrTyWY']
inversor20 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'vOuJvMaM2sgwLmb']
inversor21 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'xMbIugepa2P7lBB']
inversor22 = dtUnificadaTrativa3.loc[dtUnificadaTrativa3['SOURCE_KEY'] == 'xoJJ8DcxJEcupym']

inversores = ['4UPUqMRk7TRMgml', '81aHJ1q11NBPMrL', '9kRcWv60rDACzjR', 'Et9kgGMDl729KT4', 'IQ2d7wF4YD8zU1Q', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'Mx2yZCDsyf6DPfv', 'NgDl19wMapZy17u', 'PeE6FRyGXUgsRhN', 'Qf4GUc1pJu5T6c6', 'Quc1TzYxW2pYoWX', 'V94E5Ben1TlhnDV', 'WcxssY2VbP4hApt', 'mqwcsP2rE7J0TFp', 'oZ35aAeoifZaQzV', 'oZZkBaNadn6DNKz', 'q49J1IKaHRwDQnt', 'rrq4fwE8jgrTyWY', 'vOuJvMaM2sgwLmb', 'xMbIugepa2P7lBB', 'xoJJ8DcxJEcupym']
quantidade_de_linhas = [(inversor1.shape[0]), (inversor2.shape[0]), (inversor3.shape[0]), (inversor4.shape[0]), (inversor5.shape[0]), (inversor6.shape[0]), (inversor7.shape[0]), (inversor8.shape[0]), (inversor9.shape[0]), (inversor10.shape[0]), (inversor11.shape[0]), (inversor12.shape[0]), (inversor13.shape[0]), (inversor14.shape[0]), (inversor15.shape[0]), (inversor16.shape[0]), (inversor17.shape[0]), (inversor18.shape[0]), (inversor19.shape[0]), (inversor20.shape[0]), (inversor21.shape[0]), (inversor22.shape[0])]

cor = ['royalblue' if (inversores == 3259) else 'coral' if (inversores == 3195) else 'tomato' for inversores in quantidade_de_linhas ]
#ordem = ['royalblue', 'coral', 'tomato']

sns.barplot(x=quantidade_de_linhas, y=inversores, palette=cor)
plt.xlabel('Quantidade de linhas no conjunto de dados')
plt.savefig(diretorioImagens + '/qtdlinhasinversores.png', format='png', bbox_inches='tight')
plt.show()

#fig = px.bar(labels={"x": "Quantidade de linhas no conjunto de dados", "y": "Inversor"}, x=quantidade_de_linhas, y=inversores, color=cor, width=800)
#fig.update_yaxes(categoryorder="total ascending")

Analisando o gráfico gerado, percebe-se que nenhum inversor gerou as 3.264 linhas de dados esperadas, no entanto, a maioria (em azul) gerou 3.259 linhas de dados, o que é muito próximo do esperado. Nota-se ainda que houve um padrão em relação ao registro dos dados, sendo 12 inversores com 3.259 linhas de informação, 6 com 3.195 linhas e 4 com 2.355 linhas de dados.

Para a previsão de geração de energia fotovoltaica, serão considerados os 12 inversores que apresentaram uma maior quantidade de linhas de dados para os 34 dias.

In [ ]:
# Removendo inversores com menos de 3.259 linhas de dados da amostra
dtUnificadaTrativa4 = dtUnificadaTrativa3.drop(inversor21.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor15.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor9.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor5.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor16.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor12.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor11.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor8.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor4.index)
dtUnificadaTrativa4 = dtUnificadaTrativa4.drop(inversor1.index)

dtUnificadaTrativa4.head()

In [ ]:
dtUnificadaTrativa4.info()

In [ ]:
dtUnificadaTrativa4.shape

Sendo 12 inversores, cada um registrando informações a cada 15 minutos, se tem que em um período de 6 horas, serão necessários 576 registros de dados. O gráfico a seguir identifica em quais dias não houveram esta quantidade de registro de dados.

Percebe-se que a falta ocorreu nos dias 15, 19 e 29 de maio, e nos dias 1 e 3 de junho.

In [ ]:
# Quantidade de registros de potência DC em um período de 6 horas
fig = px.histogram(dtUnificadaTrativa4, labels={"DATE_TIME": "Data"}, x="DATE_TIME")
fig.update_yaxes(title_text='Quantidade de registros de Potência DC')

## 3.2 Potência AC e DC geradas

In [ ]:
# Potência AC e DC média gerada em um período de 6 horas
x = dtUnificadaTrativa4["DATE_TIME"]
y1 = dtUnificadaTrativa4["AC_POWER"]
y2 = dtUnificadaTrativa4["DC_POWER"]

layout = go.Layout(
    autosize=False,
    width=800,
    height=250,

    margin=go.layout.Margin(
    l=10,
    r=10,
    b=10,
    t=10,
    pad = 4
)
)

fig = go.Figure(layout = layout)
fig.add_trace(go.Histogram(histfunc="avg", y=y2, x=x, name="Potência DC"))
fig.add_trace(go.Histogram(histfunc="avg", y=y1, x=x, name="Potência AC"))

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.5)

fig.update_yaxes(title_text='Potência média (kW)')

fig.write_image(diretorioImagens + '/pot_ac_dc_media.png')

In [ ]:
# Potência AC e DC total gerada em um período de 6 horas
x = dtUnificadaTrativa4["DATE_TIME"]
y1 = dtUnificadaTrativa4["AC_POWER"]
y2 = dtUnificadaTrativa4["DC_POWER"]

layout = go.Layout(
    autosize=False,
    width=800,
    height=250,

    margin=go.layout.Margin(
    l=10,
    r=10,
    b=10,
    t=10,
    pad = 4
)
)

fig = go.Figure(layout = layout)
fig.add_trace(go.Histogram(y=y2, x=x, name="Potência DC", histfunc='sum'))
fig.add_trace(go.Histogram(y=y1, x=x, name="Potência AC", histfunc='sum'))

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.5)

fig.update_yaxes(title_text='Potência total (kW)')

fig.write_image(diretorioImagens + '/pot_ac_dc_total.png')

In [ ]:
# Perdas na geração de energia
perdas=dtUnificadaTrativa4.copy() # Copia conteúdo da tabela de dados da planta
perdas['data']=perdas['DATE_TIME'].dt.date # Reune os dados de data somente
perdas=perdas.groupby('data').sum() # Agrupa todos os dados para as 34 datas
perdas['perdas']=perdas['AC_POWER']/perdas['DC_POWER']*100 # Reune os dados de energia real convertida

perdas['perdas'].plot(style='o--',figsize=(12,5),label='Energia Real') # Plota os dados de energia real convertida

print('Perdas média: ', perdas['perdas'].mean())

plt.ylabel('Potência DC convertida (%)',fontsize=12)
plt.xlabel('Data',fontsize=12)
plt.axhline(perdas['perdas'].mean(),linestyle='--',color='green',label='Média') # Plota linha com a média dos dados de energia convertida
plt.xticks(rotation = 90)
plt.legend()
plt.savefig(diretorioImagens + '/porcentagemEnergiaConvertida.png', format='png', bbox_inches='tight')
plt.show()

## 3.3 Irradiação

In [ ]:
layout = go.Layout(
    autosize=False,
    width=800,
    height=100,

    margin=go.layout.Margin(
    l=0,
    r=0,
    b=0,
    t=0,
    pad = 0
)
)

# Irradiação média gerada em um período de 6 horas
fig = go.Figure(layout = layout)
fig = px.histogram(dtUnificadaTrativa4, labels={"DATE_TIME": "Data"}, x="DATE_TIME", y="IRRADIATION", histfunc='avg')
fig.update_yaxes(title_text='Irradiação média (W/m²)')

fig.write_image(diretorioImagens + '/irradiacao_media.png')

## 3.4 Temperatura

In [ ]:
# Temperatura do módulo média gerada em um período de 6 horas
x = dtUnificadaTrativa4["DATE_TIME"]
y1 = dtUnificadaTrativa4["AMBIENT_TEMPERATURE"]
y2 = dtUnificadaTrativa4["MODULE_TEMPERATURE"]

layout = go.Layout(
    autosize=False,
    width=800,
    height=250,

    margin=go.layout.Margin(
    l=10,
    r=10,
    b=10,
    t=10,
    pad = 4
)
)

fig = go.Figure(layout = layout)
fig.add_trace(go.Histogram(histfunc="avg", y=y1, x=x, name="Temp. Ambiente"))
fig.add_trace(go.Histogram(histfunc="avg", y=y2, x=x, name="Temp. Módulo"))

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.5)

fig.update_yaxes(title_text='Temperatura média (ºC)')
fig.write_image(diretorioImagens + '/Temperatura_media.png')

## 3.5 Simetria dos Dados

Antes de verificar a simetria das variáveis, serão criados mais dois atributos. Visto que dados de geração de energia solar dependem da sazonalidade, serão criadas as colunas "HOUR" e "DAY", de modo a identificar se estas possuem correlação com o atributo alvo, potência AC.

In [ ]:
dtUnificadaTratativa4_1 = dtUnificadaTrativa4.copy()

# Criando colunas de mês, dia e hora
dtUnificadaTratativa4_1['DAY'] = dtUnificadaTratativa4_1['DATE_TIME'].dt.day
dtUnificadaTratativa4_1['HOUR'] = dtUnificadaTratativa4_1['DATE_TIME'].dt.hour

In [ ]:
dtUnificadaTratativa4_1

In [ ]:
list(dtUnificadaTratativa4_1)

Para uma simetria perfeita, espera-se valores muito próximos de 0.

In [ ]:
print(dtUnificadaTratativa4_1.skew(), '\nSoma:', sum(abs(dtUnificadaTratativa4_1.skew())))

Os atributos de geração de energia (AC_POWER e DC_POWER) e irradiação apresentam uma simetria acima de zero, isto pode ser devido a calda-longa apresentada pelos histogramas a seguir.

In [ ]:
# Gerar um histograma para cada atributo numérico
dtSimetria = dtUnificadaTratativa4_1.drop(['DATE_TIME', 'DAY', 'HOUR'], axis = 1)
dtSimetria.hist(bins=50, figsize=(10,30), layout=(7,2))
plt.savefig(diretorioImagens + '/simetria_1.png', format='png', bbox_inches='tight')
plt.show()

Percebe-se que na variável de geração de energia (AC_POWER) e para a variável de irradiação, se tem uma tail-havy (calda-longa) no valor nulo, isto se dá pois durante a noite não existe irradiação solar e consequentemente não se tem geração de energia.

O gráfico a seguir, mostra que para todos os inversores, no período entre as 19h e 5h, o valor é nulo, logo, não será necessário considerar este intervalo na previsão.

In [ ]:
inversores = dtUnificadaTratativa4_1.copy()
inversores['Horário'] = inversores['DATE_TIME'].dt.hour # Retorna um array com os dados de hora
inversores.set_index('Horário').groupby('SOURCE_KEY')['AC_POWER'].plot(style='o', legend=True, figsize=(13,8))

#plt.title('Potência AC Gerada durante o dia para todos os inversores', size=17)
plt.ylabel('Potência AC (kW)', fontsize=15)
plt.legend()
plt.savefig(diretorioImagens + '/energia_AC_total.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
# Removendo intervalo onde a geração de enegia é nula
filtro = (dtUnificadaTratativa4_1['DATE_TIME'].dt.hour > 5) & (dtUnificadaTratativa4_1['DATE_TIME'].dt.hour < 19)

dtUnificadaTratativa4_2 = dtUnificadaTratativa4_1[filtro]

dtUnificadaTratativa4_2.describe()

In [ ]:
inversores = dtUnificadaTratativa4_2.copy()
inversores['Horário'] = inversores['DATE_TIME'].dt.hour # Retorna um array com os dados de hora
inversores.set_index('Horário').groupby('SOURCE_KEY')['AC_POWER'].plot(style='o', legend=True, figsize=(13,10))

plt.ylabel('Potência AC (kW)', fontsize=17)
plt.legend()
plt.savefig(diretorioImagens + '/energia_AC_total_6-18.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
# Gerar um histograma para cada atributo numérico
dtSimetria = dtUnificadaTratativa4_2.drop(['DATE_TIME', 'DAY', 'HOUR'], axis = 1)
dtSimetria.hist(bins=50, figsize=(10,30), layout=(7,2))
plt.savefig(diretorioImagens + '/simetria_2.png', format='png', bbox_inches='tight')
plt.show()

Mesmo removendo o intervalo onde a geração de energia deveria ser nula, percebe-se que os histrogramas ainda apresentam uma calda-longa. O que pode indicar um mau funcionamento dos inversores. Serão reunidos os valores nulos em uma amostra, para identificar o período em que estes ocorrem e se a irradiação possui influência sobre estes valores.

In [ ]:
# Reunir os valores nulos de energia
dataframe = dtUnificadaTratativa4_2.copy()
filtro = dtUnificadaTratativa4_2['AC_POWER'] == 0

geracaoACNula = dataframe[filtro]

In [ ]:
# Histograma com as quantidade de valores nulos 
diasNulos = geracaoACNula['DATE_TIME'].dt.hour

fig = px.histogram(geracaoACNula, labels={"x": "Data"}, x=diasNulos, y="AC_POWER", histfunc='count')
fig.update_yaxes(title_text='Quantidade de valores nulos durante os 34 dias')

In [ ]:
# Histograma com a média de irradiação para quando a geração de energia foi nula
diasNulos = geracaoACNula['DATE_TIME'].dt.hour

layout = go.Layout(
    autosize=False,
    width=800,
    height=250,

    margin=go.layout.Margin(
    l=0,
    r=0,
    b=0,
    t=0,
    pad = 0
)
)

fig = go.Figure(layout = layout)
fig = px.histogram(geracaoACNula, labels={'x': "Hora"}, x=diasNulos, y="IRRADIATION", histfunc='avg')
fig.update_yaxes(title_text='Irradiação solar média para os valores nulos (W/m²)')

fig.write_image(diretorioImagens + '/irradiacao_valores_nulos.png')

Pode-se ver que a irradiação seguiu um padrão normal, ou seja, os valores nulos de geração de energia apresentados entre as 7h e 17h estão incorretos, o que pode indicar um mau funcionamento dos inversores, ou pode ter sido um período em que foi feita alguma manutenção e o sistema foi desligado. Na figura a seguir fica evidente o comportamento inesperado para alguns dos 12 inversores.

Os valores nulos de geração de energia neste intervalo serão substituídos pelo seu valor antecedente, o que irá se aproximar do valor gerado naquele instante.

### 3.5.1 Potência AC gerada para cada um dos 34 dias sem tratamento para nulos

In [ ]:
geracao=dtUnificadaTratativa4_2.copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-05-18')]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(3,1),figsize=(10,12), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-05-18')]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(3,1),figsize=(10,12), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_com_nulos_1.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-18')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-18')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-05-22'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-18')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-05-22'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_com_nulos_2.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-22')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-22')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-05-26'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-22')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-05-26'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_com_nulos_3.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-26')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-26')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-05-30'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-26')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-05-30'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_com_nulos_4.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-30')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-30')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-03'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-05-30')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-03'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_com_nulos_5.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-03')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-03')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-07'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-03')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-07'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_com_nulos_6.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-07')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-07')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-11'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-07')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-11'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/\energia_AC_inversores_com_nulos_7.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-11')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-11')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-15'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-11')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-15'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_com_nulos_8.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-15')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-15')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-18'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(3,1),figsize=(10,12), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTratativa4_2.loc[dtUnificadaTratativa4_2['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTratativa4_2['DATE_TIME'] > ('2020-06-15')) & (dtUnificadaTratativa4_2['DATE_TIME'] < ('2020-06-18'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(3,1),figsize=(10,12), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_com_nulos_9.png', format='png', bbox_inches='tight')
plt.show()

### 3.5.2 Tratamento para nulos

In [ ]:
# Coletar amostra no intervalo das 7h às 17h
filtro = (dtUnificadaTratativa4_2['DATE_TIME'].dt.hour >= 7) & (dtUnificadaTratativa4_2['DATE_TIME'].dt.hour <= 17)

dtGeracaoACIncorreta = dtUnificadaTratativa4_2[filtro]

In [ ]:
# Histograma com a média de irradiação para quando a geração de energia foi nula incorretamente
diasNulos = dtGeracaoACIncorreta['DATE_TIME'].dt.hour

fig = px.histogram(dtGeracaoACIncorreta, labels={'x': "Hora"}, x=diasNulos, y="IRRADIATION", histfunc='avg')
fig.update_yaxes(title_text='Irradiação solar média')

In [ ]:
# Substituir valores nulos por NaN
dtGeracaoACIncorretaNAN = dtGeracaoACIncorreta.copy()
geracaoACIncorretaNAN = dtGeracaoACIncorretaNAN[['AC_POWER', 'DC_POWER']].replace(0, np.nan)

# Exibe a quantidade de valores NaN
geracaoACIncorretaNAN.isnull().sum()

In [ ]:
dtGeracaoACIncorretaNAN

In [ ]:
geracaoACIncorretaNAN.head(1000)

In [ ]:
# Substituindo valor NaN com o valor seguinte AC
newColumnAC = geracaoACIncorretaNAN['AC_POWER'].fillna(method='bfill')
newColumnDC = geracaoACIncorretaNAN['DC_POWER'].fillna(method='bfill')

newColumnAC.head(1000)

In [ ]:
newColumnDC.head(1000)

In [ ]:
# Substitui a coluna com valor nulo pela coluna com valor numérico
dtGeracaoACCorrigida = dtGeracaoACIncorreta.copy()

# Retira a coluna de geração incorreta
dtGeracaoACCorrigida = dtGeracaoACCorrigida.drop(['AC_POWER', 'DC_POWER'], axis = 1)

# Insere a coluna de geração corrigida na amostra
dtGeracaoACCorrigida['AC_POWER'] = newColumnAC
dtGeracaoACCorrigida['DC_POWER'] = newColumnDC

dtGeracaoACCorrigida.head(1000)

In [ ]:
dtGeracaoACCorrigida.info()

In [ ]:
dtGeracaoACCorrigida

In [ ]:
dtUnificadaTratativa4_2.info()

In [ ]:
dtUnificadaTratativa4_2

In [ ]:
# Coletar amostra diferente do intervalo das 7h às 17h
filtro = (dtUnificadaTratativa4_2['DATE_TIME'].dt.hour < 7) | (dtUnificadaTratativa4_2['DATE_TIME'].dt.hour > 17)

dtTemp4_2 = dtUnificadaTratativa4_2.copy()
dtTemp4_2 = dtTemp4_2[filtro]

dtTemp4_2.info()

In [ ]:
dtTemp4_2

In [ ]:
# Une as linhas com os valores nulos corrigidos com a amostra original
dtUnificadaTrativa4_3 = pd.merge(dtTemp4_2, dtGeracaoACCorrigida, how = 'outer')

dtUnificadaTrativa4_3.info()

In [ ]:
dtUnificadaTrativa4_3

In [ ]:
# Gerar um histograma para cada atributo numérico
dtSimetria = dtUnificadaTrativa4_3.drop(['DATE_TIME', 'DAY', 'HOUR'], axis = 1)
dtSimetria.hist(bins=50, figsize=(10,30), layout=(7,2))
plt.savefig(diretorioImagens + '/simetria_3.png', format='png', bbox_inches='tight')
plt.show()

Agora, é perceptível que a calda-longa que aparece no atributo de geração de energia, se assemelha com a calda-longa do atributo de irradiação, o que faz sentido, quando a irradiação for igual a zero, a geração de energia tende a ser igual a zero.

In [ ]:
print(dtUnificadaTrativa4_3.skew(), '\nSoma:', sum(abs(dtUnificadaTrativa4_3.skew())))

### 3.5.3 Potência AC gerada para cada um dos 34 dias com tratamento para nulos

In [ ]:
geracao=dtUnificadaTrativa4_3.copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-05-18')]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(3,1),figsize=(10,12), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-05-18')]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(3,1),figsize=(10,12), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=12)
        ax[linha,coluna].set_ylabel('Energia AC (kW)')
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_1.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-18')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-18')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-05-22'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-18')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-05-22'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_2.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-22')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-22')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-05-26'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-22')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-05-26'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_3.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-26')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-26')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-05-30'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-26')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-05-30'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_4.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-30')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-30')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-03'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-05-30')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-03'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_5.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-03')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-03')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-07'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-03')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-07'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_6.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-07')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-07')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-11'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-07')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-11'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_7.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-11')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-11')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-15'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-11')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-15'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(4,1),figsize=(10,15), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_8.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geracao=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-15')].copy()

geracao['Horário']=geracao['DATE_TIME'].dt.hour
geracao['Data']=geracao['DATE_TIME'].dt.date

# just for columns
cols=geracao.groupby(['Horário','Data'])['AC_POWER'].mean().unstack()

inversores = ['xoJJ8DcxJEcupym', 'WcxssY2VbP4hApt', '9kRcWv60rDACzjR', 'vOuJvMaM2sgwLmb', 'rrq4fwE8jgrTyWY', 'LYwnQax7tkwH5Cb', 'LlT2YUhhzqhg5Sw', 'q49J1IKaHRwDQnt', 'oZZkBaNadn6DNKz', 'PeE6FRyGXUgsRhN', '81aHJ1q11NBPMrL', 'V94E5Ben1TlhnDV']
cores = ['royalblue', 'tomato', 'gold', 'lightgreen', 'plum', 'lightgray', 'blue', 'red', 'yellow', 'green', 'black', 'brown']

inversores3259=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']=='xoJJ8DcxJEcupym'].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-15')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-18'))]
inversores3259['Horário']=inversores3259['DATE_TIME'].dt.hour
inversores3259['Data']=inversores3259['DATE_TIME'].dt.date

ax=inversores3259.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(3,1),figsize=(10,12), color=cores[0])

j=1
for j in range(1,12):
    inversores3259_2=dtUnificadaTrativa4_3.loc[dtUnificadaTrativa4_3['SOURCE_KEY']==inversores[j]].loc[(dtUnificadaTrativa4_3['DATE_TIME'] > ('2020-06-15')) & (dtUnificadaTrativa4_3['DATE_TIME'] < ('2020-06-18'))]
    inversores3259_2['Horário']=inversores3259_2['DATE_TIME'].dt.hour
    inversores3259_2['Data']=inversores3259_2['DATE_TIME'].dt.date
    inversores3259_2.groupby(['Horário','Data'])['AC_POWER'].mean().unstack().plot(sharex=True,subplots=True,layout=(3,1),figsize=(10,12), color=cores[j], ax=ax)
    j=j+1

i=0
for linha in range(len(ax)):
    for coluna in range(len(ax[linha])):
        ax[linha,coluna].set_title(cols.columns[i],size=15)
        ax[linha,coluna].set_ylabel('Potência AC (kW)', size=15)
        ax[linha,coluna].legend(inversores, bbox_to_anchor = (1.02, 1))
        i=i+1

plt.tight_layout()
plt.savefig(diretorioImagens + '/energia_AC_inversores_sem_nulos_9.png', format='png', bbox_inches='tight')
plt.show()

Com as trativas realizadas, obtivemos uma simetria muito melhor para todos os atributos, agora todos estão próximos de zero, quanto mais próximo de zero, mais simétrica é a distribuição dos dados.

## 3.6 Correlação

In [ ]:
# Posiciona o atributo a ser previsto na última coluna do dataset
colunaACPower = dtUnificadaTrativa4_3['AC_POWER']
dtUnificadaTratativa4_4 = dtUnificadaTrativa4_3.copy()
dtUnificadaTratativa4_4 = dtUnificadaTratativa4_4.drop(['AC_POWER'], axis=1)
dtUnificadaTratativa4_4["AC_POWER"] = colunaACPower

dtUnificadaTratativa4_4

In [ ]:
# Cria coluna "AC_POWER_FUTURO"
dtUnificadaTratativa4_4["AC_POWER_FUTURO"] = colunaACPower.shift(-1)
dtUnificadaTratativa4_4 = dtUnificadaTratativa4_4.dropna()

dtUnificadaTratativa4_4

In [ ]:
# Remover atributos DAILY_YIELD e TOTAL_YIELD

dtUnificadaTratativa4_5 = dtUnificadaTratativa4_4.copy()
dtUnificadaTratativa4_5 = dtUnificadaTratativa4_5.drop(['DAILY_YIELD', 'TOTAL_YIELD'], axis=1)

dtUnificadaTratativa4_5

A seguir será visto o quanto cada atributo se correlaciona com a energia gerada (AC_POWER). Percebe-se uma forte correlação positiva, tirando a DC_POWER, na irradiação e na temperatura do módulo, o que faz sentido, visto que quanto maior a irradiação solar, maior deve ser a energia gerada, e quanto maior a irradiação sobre os módulos, maior será sua temperatura. Os atributos de rendimento e tempo por sua vez, mostram que não existe uma correlação linear com a geração de energia, visto que os coeficientes são muito próximos de zero.

A baixa correlação para as variáveis de tempo, pode se dar pela baixa quantidade de dados da amostra.

In [ ]:
corr_matrix = dtUnificadaTratativa4_5.corr()

corr_matrix["AC_POWER_FUTURO"].sort_values(ascending=False)

In [ ]:
fig = plt.figure(figsize = (10, 10))

sns.heatmap(dtUnificadaTratativa4_4.corr(method = 'pearson'), annot = True, square = True)
plt.show()
fig.savefig(diretorioImagens + '/corr_all.png', format='png', bbox_inches='tight')

No gráfico acima, percebe-se que a colinearidade entre os atributos de temperatura são muito próximas de 1, o que pode ser um forte indício de multicolinearidade, ou seja, dois atributos que representam a mesma coisa. Fica evidente que o atributo de potência DC é multicolinear com o atributo de potência AC, mas isto já era perceptível com a análise gráfica de geração de energia.

Visto que os atributos de rendimento não apresentam influência no atributo de geração de energia, supõe-se que estes serão excluídos da amostra de dados, mas antes, será realizada uma análise de multicolinearidade entre as variáveis independentes.

### 3.7.1 Multicolinearidade

Na multicolinearidade se tem atributos preditores que se correlacionam com outros preditores, são atributos redundantes na amostra de dados.

In [ ]:
dtX = dtUnificadaTratativa4_5.copy()
dtX = dtX.drop(['DATE_TIME', 'SOURCE_KEY', 'DC_POWER', 'AC_POWER_FUTURO'], axis = 1)

# Gerando matriz de correlação e recombinando
corr = np.corrcoef(dtX, rowvar = 0)
autovalor, autovetor = np.linalg.eig(corr)

menor = 999
index = 0
for i, val in enumerate(autovalor):
    if val < menor:
        menor = val
        index = i

print('Menor autovalor:', menor, 'Index:', index)

In [ ]:
menorAutovetor = abs(autovetor[:, 5])

for i, val in enumerate(autovetor[:, 5]):
    print('Variável', i,':', abs(val))

In [ ]:
# Exibir atributos com o maior valor absoluto para o autovetor
colunas = dtX.columns

colunas[[5, 2, 0, 1]]

Atributos com valores VIF próximos de 1, indicam que não existe correlação entre eles, a seguir é calculado o valor VIF para todos os atributos independentes da amostra.

In [ ]:
def calcular_VIF(X):
    vif = pd.DataFrame()
    vif['Atributos'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    
    return vif

calcular_VIF(dtX)

Percebe-se um alto valor para os atributos de temperatura e irradiação. Primeiramente será removido o atributo 'AMBIENT_TEMPERATURE' e em seguida o VIF será novamente calculado.

In [ ]:
dtX = dtUnificadaTratativa4_5.copy()
dtX = dtX.drop(['DATE_TIME', 'SOURCE_KEY', 'DC_POWER', 'AMBIENT_TEMPERATURE', 'AC_POWER_FUTURO'], axis = 1)

calcular_VIF(dtX)

In [ ]:
dtX = dtUnificadaTratativa4_5.copy()
dtX = dtX.drop(['DATE_TIME', 'SOURCE_KEY', 'DC_POWER', 'AMBIENT_TEMPERATURE', 'AC_POWER_FUTURO', 'MODULE_TEMPERATURE'], axis = 1)

calcular_VIF(dtX)

In [ ]:
dtX = dtUnificadaTratativa4_5.copy()
dtX = dtX.drop(['DATE_TIME', 'SOURCE_KEY', 'DC_POWER', 'AMBIENT_TEMPERATURE', 'AC_POWER_FUTURO', 'MODULE_TEMPERATURE', 'AC_POWER'], axis = 1)

calcular_VIF(dtX)

## 3.7 Relatório OLS

In [ ]:
# Carregando atributos com exceção do atributo alvo
X = dtUnificadaTratativa4_5.drop(['DATE_TIME', 'SOURCE_KEY', 'AC_POWER_FUTURO', 'DC_POWER'], axis = 1)
Xc = sm.add_constant(X)

y = dtUnificadaTratativa4_5['AC_POWER_FUTURO'].values

# Criando e treinando modelo
modelo = sm.OLS(y, Xc)
modelo1 = modelo.fit()

# Visualizando resumo do modelo
modelo1.summary()

O modelo explica 92% da variância dos dados.

Percebe-se que os atributos HOUR e MODULE_TEMPERATURE apresentaram um valor muito baixo para t, o que indica uma baixa relevância para o modelo preditivo.

In [ ]:
# Carregando atributos com exceção do atributo alvo
X = dtUnificadaTratativa4_5.drop(['DATE_TIME', 'SOURCE_KEY', 'DC_POWER', 'AMBIENT_TEMPERATURE', 'AC_POWER_FUTURO', 'MODULE_TEMPERATURE', 'AC_POWER'], axis = 1)
Xc = sm.add_constant(X)

y = dtUnificadaTratativa4_5['AC_POWER_FUTURO'].values

# Criando e treinando modelo
modelo = sm.OLS(y, Xc)
modelo2 = modelo.fit()

# Visualizando resumo do modelo
modelo2.summary()

In [ ]:
# Carregando atributos com exceção do atributo alvo
X = dtUnificadaTratativa4_5.drop(['DATE_TIME', 'SOURCE_KEY', 'DC_POWER', 'AMBIENT_TEMPERATURE', 'AC_POWER_FUTURO', 'MODULE_TEMPERATURE', 'AC_POWER', 'DAY', 'HOUR'], axis = 1)
Xc = sm.add_constant(X)

y = dtUnificadaTratativa4_5['AC_POWER_FUTURO'].values

# Criando e treinando modelo
modelo = sm.OLS(y, Xc)
modelo2 = modelo.fit()

# Visualizando resumo do modelo
modelo2.summary()

In [ ]:
# Excluir atributos independentes descartados com a análise de correlação e multicolinearidade

dtUnificadaTratativa5 = dtUnificadaTratativa4_5.drop(['DC_POWER'], axis = 1)
dtTratadaCompleta = dtUnificadaTratativa5.copy()

dtUnificadaTratativa5.describe()

In [ ]:
dtTratadaCompleta.describe()

In [ ]:
dtUnificadaTratativa5["SOURCE_KEY"].value_counts()

## 3.8 Outliers

In [ ]:
def boxplot_individuais(data, colunas, largura = 10, altura = 8):
    fig = plt.figure()
    fig.subplots_adjust(hspace = 0.4, wspace = 0.4)
    fig.set_figheight(altura)
    fig.set_figwidth(largura)
    
    ajusteColunas = math.ceil(len(colunas) / 2)
    
    for i, coluna in enumerate(colunas):
        ax = fig.add_subplot(ajusteColunas, 2, i + 1)
        sns.boxplot(x = data[coluna])
        
    plt.tight_layout()
    plt.savefig(diretorioImagens + '/outliers.png')  
    plt.show()

Pode-se ver que nenhum atributo apresenta valores extremos, que fujam muito do padrão.

In [ ]:
# Separa os valores qualitativos da amostra
qualitativos = dtUnificadaTratativa5.drop(["AC_POWER", "IRRADIATION"], axis = 1).columns

# Retira valores qualitativos da amostra (Data e número de identificação dos inversores)
quantitativos = dtUnificadaTratativa5.drop(['DATE_TIME', 'SOURCE_KEY', 'AC_POWER_FUTURO', "HOUR", "DAY"], axis = 1).columns

boxplot_individuais(dtUnificadaTratativa5, quantitativos[0:5])

## 3.9 Normalização

In [ ]:
dtUnificadaTratativa5[quantitativos]

In [ ]:
dtTratadaCompleta

In [ ]:
#dtNormalizada.to_csv(diretorioDados + '/planilha_tratada_normalizada_completa.csv')
dtTratadaCompleta.to_csv(diretorioDados + '/planilha_tratada_completa_2.csv')

In [ ]:
#dtUnificadaTrativaNormalizadaCompleta.to_csv(diretorioDados + '/planilha_tratada_normalizada.csv')
#dtUnificadaTratativa5.to_csv(diretorioDados + '/planilha_tratada.csv')